In [1]:
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, DateTime
from sqlalchemy.orm import sessionmaker, scoped_session
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship, backref
import numpy as np
import pandas as pd
import datetime



#importing items
df_items = pd.read_csv("movielens/items_amazon_musical_instruments.csv", low_memory= False)

# extracting specific columns
df_items = df_items[["asin", "title", "description", "imUrl"]]
df_items[["old_id"]] = df_items[['asin']]
# renaming columns
df_items = df_items.rename(columns={'asin': 'id', 'title': 'title', 'description': 'description', 'imUrl': 'imgurl'})


import config
from sqlalchemy.ext.declarative import declarative_base

# creating keys
df_items = df_items.sort_values("id")

#creating key for items
df_items['item_key'] = range(len(df_items))

#adding 1 to the range
# df_items['item_key'] = df_items['item_key'] + 1


df_items = df_items[["item_key", "title", "description", "imgurl","old_id"]]
df_items = df_items.rename(columns={'item_key': 'id'})
# df_items['id'] = df_items['id'] + 1


df_items = df_items.sort_values("id")

#removing the top row since its not an item
df_items = df_items.iloc[1:]

df_items_key = df_items[["old_id", "id"]]
df_items_key = df_items_key.rename(columns={'id': 'item_key'})


In [2]:
df_items.head()

,id,title,description,imgurl,old_id
25,1,The Zen of Screaming 2 (DVD),Alfred is pleased to distribute the second rel...,http://ecx.images-amazon.com/images/I/51Wyfhbx...,073904852X
58,2,niceEshop Studio Microphone Mic Wind Screen Po...,New generic Studio Microphone Mic Wind Screen ...,http://ecx.images-amazon.com/images/I/510D2cVp...,1384719342
1,3,Double Concerto in D Minor By Johann Sebastian...,Composer: J.S. Bach.Peters Edition.For two vio...,http://ecx.images-amazon.com/images/I/41m6ygCq...,14072149
59,4,Brokeback Mountain,Two cowboys (Heath Ledger & Jake Gyllenhaal) i...,http://ecx.images-amazon.com/images/I/517Ttoaa...,1417030321
60,5,Musicians Institute: Home Recording Basics,NaN,http://ecx.images-amazon.com/images/I/518PXBD9...,1423403363


In [3]:
df_items_key.head()

,old_id,item_key
25,073904852X,1
58,1384719342,2
1,14072149,3
59,1417030321,4
60,1423403363,5


In [4]:
#Importing Ratings
# rating_colnames = ['user_id', 'item_id', 'rating','timestamp']
# df_ratings = pd.read_table("movielens\\u.data", names = rating_colnames)
df_ratings = pd.read_csv("movielens/reviews_amazon_musical_instruments_small.csv")

# subsetting dataframe
df_ratings = df_ratings[["reviewerID", "asin", "overall", "unixReviewTime"]]
df_ratings.unixReviewTime = pd.to_datetime(df_ratings["unixReviewTime"],unit = 's')

#changing column names
df_ratings = df_ratings.rename(columns={'reviewerID': 'user_id', 'asin': 'item_id', 'overall': 'rating', 'unixReviewTime': 'timestamp'})



# creating keys
df_ratings = df_ratings.sort_values("user_id")

# df_ratings["old_user_id"] = df_ratings["user_id"]

In [89]:
df_ratings.shape

(18139, 5)

In [90]:
item_filter = pd.DataFrame(df_ratings['item_id'])
item_filter = item_filter.groupby('item_id').size()
item_filter = item_filter.to_frame().reset_index()
item_filter.columns.values[1] = 'count'
item_filter = item_filter.sort_values('count', ascending=False)
item_filter = item_filter.head(n=1000)
item_filter = item_filter[['item_id']]

In [91]:
df_ratings = pd.merge(left=df_ratings,right=item_filter, left_on='item_id', right_on='item_id')
df_ratings.shape

(13326, 5)

In [5]:
df_ratings.head()

,user_id,item_id,rating,timestamp
12004,A060341735DGIVP8HTQIM,B00005YBML,5,2012-11-12
16969,A08404673O0MPFFC8XZZR,B00006OALO,5,2012-11-23
10101,A08994231UD3NPD71GV1X,B000059PEG,5,2013-03-22
2353,A09151771XCBU491CYA7Q,B000000AKC,3,2013-07-01
13116,A10044ECXDUVKS,B000068NUT,5,2013-03-19


In [6]:

df_ratings.user_id = df_ratings.user_id.astype("category")

df_ratings["userid_key"] = df_ratings["user_id"].cat.codes
df_ratings["userid_key"] = df_ratings["userid_key"] + 1

# df_ratings['userid_key'] = range(len(df_ratings))

#creating key for items
df_ratings_key = df_ratings[["userid_key", "user_id"]]
# df_ratings_key['userid_key'] = df_ratings_key['userid_key'] + 1

In [7]:
df_ratings_key.head()

,userid_key,user_id
12004,1,A060341735DGIVP8HTQIM
16969,2,A08404673O0MPFFC8XZZR
10101,3,A08994231UD3NPD71GV1X
2353,4,A09151771XCBU491CYA7Q
13116,5,A10044ECXDUVKS


In [8]:
df_ratings.head()

,user_id,item_id,rating,timestamp,userid_key
12004,A060341735DGIVP8HTQIM,B00005YBML,5,2012-11-12,1
16969,A08404673O0MPFFC8XZZR,B00006OALO,5,2012-11-23,2
10101,A08994231UD3NPD71GV1X,B000059PEG,5,2013-03-22,3
2353,A09151771XCBU491CYA7Q,B000000AKC,3,2013-07-01,4
13116,A10044ECXDUVKS,B000068NUT,5,2013-03-19,5


In [9]:
# df_ratings['user_id']=df_ratings['user_id']+1
# df_ratings['item_id'] = df_ratings['item_id']+1

# combining columns with rating ids
df_ratings = pd.merge(df_ratings, df_items_key, left_on = "item_id", right_on="old_id")

In [10]:
df_ratings.head()

,user_id,item_id,rating,timestamp,userid_key,old_id,item_key
0,A060341735DGIVP8HTQIM,B00005YBML,5,2012-11-12,1,B00005YBML,3206
1,A3E6G367XXV814,B00005YBML,5,2014-02-17,10173,B00005YBML,3206
2,A08404673O0MPFFC8XZZR,B00006OALO,5,2012-11-23,2,B00006OALO,3860
3,A25P3PGQLE2MPR,B00006OALO,5,2005-09-26,4925,B00006OALO,3860
4,A2SA2R4URY0F9Z,B00006OALO,5,2011-06-19,7497,B00006OALO,3860


In [11]:
#rearrange and rename columns
# df_ratings = df_ratings[["userid_key", "item_key", "rating", "user_id"]]
df_ratings = df_ratings[["userid_key", "item_key", "rating", 'timestamp', "user_id"]]
df_ratings = df_ratings.rename(columns={'user_id': 'old_user_id'})


# df_ratings = df_ratings.rename(columns={'userid_key': 'user_id_new', 'item_key': 'item_id'})
df_ratings = df_ratings.rename(columns={'userid_key': 'user_id', 'item_key': 'item_id'})

df_ratings = df_ratings.sort_values("user_id")






In [12]:
df_ratings.head()

,user_id,item_id,rating,timestamp,old_user_id
0,1,3206,5,2012-11-12,A060341735DGIVP8HTQIM
2,2,3860,5,2012-11-23,A08404673O0MPFFC8XZZR
6,3,2665,5,2013-03-22,A08994231UD3NPD71GV1X
9,4,182,3,2013-07-01,A09151771XCBU491CYA7Q
12,5,3490,5,2013-03-19,A10044ECXDUVKS


In [13]:
df_users = pd.DataFrame()

df_users["id"] = df_ratings.user_id.unique()
df_users = pd.merge(df_users,df_ratings , left_on="id", right_on= "user_id")
# df_users["id"] = df_users["id"] + 1

df_users = df_users[['id', 'old_user_id']]

In [14]:
df_users.head()

,id,old_user_id
0,1,A060341735DGIVP8HTQIM
1,2,A08404673O0MPFFC8XZZR
2,3,A08994231UD3NPD71GV1X
3,4,A09151771XCBU491CYA7Q
4,5,A10044ECXDUVKS


In [ ]:



#creating data format
# df_ratings = df_ratings[['user_id', 'item_id', 'rating']]

#Generating Sample
# df_ratings = df_ratings.sample(frac=0.05, replace=True)

#reading item data
# item_colnames = ['id', 'title']
# df_items = pd.read_table("movielens/u.item", sep = '|', usecols =[0,1], names = item_colnames)

# df_items['title'] = df_items['title'].apply(lambda x: x.decode('unicode_escape').encode('ascii', 'ignore').strip())


#Importing Users
# user_colnames = ['id', 'age', 'gender','occupation', 'zipcode']
# df_users = pd.read_table("movielens/u.user", sep = '|', names = user_colnames)


#reading item data
# item_colnames = ['id', 'title']
# df_items = pd.read_table("movielens/u.item", sep = '|', usecols =[0,1], names = item_colnames)

# df_items['title'] = df_items['title'].apply(lambda x: x.decode('unicode_escape').encode('ascii', 'ignore').strip())


#Importing Users
# user_colnames = ['id', 'age', 'gender','occupation', 'zipcode']
# df_users = pd.read_table("movielens/u.user", sep = '|', names = user_colnames)

#creating data format
# df_users = df_users[['id', 'age', 'gender', 'zipcode']]


#connect to the database

engine = create_engine(config.DB_URI, echo=False)
session = scoped_session(sessionmaker(bind=engine,
                                      autocommit = False,
                                      autoflush = False))




In [16]:
df_ratings.to_csv('ccol.csv')

In [9]:
# Append items
df_users.to_sql('users',engine,if_exists='append', index=False) #if_exists='append'
session.commit()


In [ ]:
# Append ratings
df_items.to_sql('items',engine,if_exists='append', index=False)#if_exists='append'
session.commit()

df_ratings.to_sql('ratings',engine,if_exists='append', index=False)#if_exists='append'
session.commit()

In [156]:
#reading item data
# item_colnames = ['id', 'title']
# df_items = pd.read_table("movielens/u.item", sep = '|', usecols =[0,1], names = item_colnames)

# df_items['title'] = df_items['title'].apply(lambda x: x.decode('unicode_escape').encode('ascii', 'ignore').strip())


#Importing Users
# user_colnames = ['id', 'age', 'gender','occupation', 'zipcode']
# df_users = pd.read_table("movielens/u.user", sep = '|', names = user_colnames)

df_users = pd.DataFrame()

df_users["id"] = df_ratings.user_id.unique()




#reading item data
# item_colnames = ['id', 'title']
# df_items = pd.read_table("movielens/u.item", sep = '|', usecols =[0,1], names = item_colnames)

# df_items['title'] = df_items['title'].apply(lambda x: x.decode('unicode_escape').encode('ascii', 'ignore').strip())


#Importing Users
# user_colnames = ['id', 'age', 'gender','occupation', 'zipcode']
# df_users = pd.read_table("movielens/u.user", sep = '|', names = user_colnames)

#creating data format
# df_users = df_users[['id', 'age', 'gender', 'zipcode']]


#connect to the database

engine = create_engine(config.DB_URI, echo=False)
session = scoped_session(sessionmaker(bind=engine,
                                      autocommit = False,
                                      autoflush = False))


# Append items
df_users.to_sql('users',engine,if_exists='append', index=False) #if_exists='append'
session.commit()

# Append ratings
df_items.to_sql('items',engine,if_exists='append', index=False)#if_exists='append'
session.commit()

df_ratings.to_sql('ratings',engine,if_exists='append', index=False)#if_exists='append'
session.commit()

C:\Users\Nabeel\Anaconda2\lib\site-packages\pymysql\cursors.py:165: Warning: (1287, u"'@@tx_isolation' is deprecated and will be removed in a future release. Please use '@@transaction_isolation' instead")
  result = self._query(query)


IntegrityError: (pymysql.err.IntegrityError) (1062, u"Duplicate entry '1' for key 'PRIMARY'") [SQL: u'INSERT INTO users (id) VALUES (%(id)s)'] [parameters: ({'id': 0L}, {'id': 1L}, {'id': 2L}, {'id': 3L}, {'id': 4L}, {'id': 5L}, {'id': 6L}, {'id': 7L}  ... displaying 10 of 15949 total bound parameter sets ...  {'id': 15947L}, {'id': 15948L})]

In [155]:
df_users

,id
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9


In [42]:


#reading item data
# item_colnames = ['id', 'title']
# df_items = pd.read_table("movielens/u.item", sep = '|', usecols =[0,1], names = item_colnames)

# df_items['title'] = df_items['title'].apply(lambda x: x.decode('unicode_escape').encode('ascii', 'ignore').strip())


#Importing Users
# user_colnames = ['id', 'age', 'gender','occupation', 'zipcode']
# df_users = pd.read_table("movielens/u.user", sep = '|', names = user_colnames)

#creating data format
# df_users = df_users[['id', 'age', 'gender', 'zipcode']]


#connect to the database

engine = create_engine(config.DB_URI, echo=False)
session = scoped_session(sessionmaker(bind=engine,
                                      autocommit = False,
                                      autoflush = False))


# Append items
df_users.to_sql('users',engine,if_exists='append', index=False) #if_exists='append'
session.commit()

# Append ratings
df_items.to_sql('items',engine,if_exists='append', index=False)#if_exists='append'
session.commit()

df_ratings.to_sql('ratings',engine,if_exists='append', index=False)#if_exists='append'
session.commit()


KeyError: "['user_id' 'item_id' 'rating'] not in index"